In [1]:
#### STANDART MODULES INITIALISING
### Importing standard modules and date-special modules:
import numpy as np
import pandas as pd
import math
from datetime import date

In [2]:
### Constants declaring:
#path_msci_data = 'Data_Files/Source_Files/sample_data.xlsx'
path_msci_data = 'Data_Files/Source_Files/sample_data_gaps.xlsx'
tab_monthly = 'monthly_data'
tab_daily = 'daily_returns'
tab_ivol = 'ivol_data'
tab_map = 'country_map'

In [3]:
### Extracting universe data:
df_universe = pd.read_excel(io = path_msci_data, sheet_name = tab_monthly, skiprows = [0, 2], header = 0)
df_universe = df_universe.loc[:, ['dates', 'region', 'ctry']]
df_universe.columns = ['Date', 'Market', 'Code']
df_universe.set_index(['Code', 'Date'], inplace = True)
ser_universe = df_universe.squeeze()
ser_universe.sort_index(level = [0, 1], inplace = True)
ser_universe.replace({50 : 'DM', 57 : 'EM', 504 : 'FM'}, inplace = True)

In [4]:
### Filtering market universe:
arr_markets_needed = ['DM', 'FM', 'EM']
ser_market_membership = ser_universe[ser_universe.isin(arr_markets_needed)]
index_market = ser_market_membership.index

In [5]:
### Extracting returns data:
df_returns = pd.read_excel(io = path_msci_data, sheet_name = tab_daily, skiprows = [0, 2], header = 0)
df_returns = df_returns.loc[:, ['dates', 'ctry', 'retusd', 'retloc']]
df_returns.columns = ['Date', 'Code', 'Ret USD', 'Ret LOC']
df_returns.set_index(['Code', 'Date'], inplace = True)
df_returns.sort_index(level = [0, 1], inplace = True)

In [22]:
ser_realized_ret_LOC = df_returns['Ret LOC'].copy()
#print('Ret from xlsx:', ser_realized_ret_LOC.loc['AU', '2002-01-31' : '2007-01-31'].head())
### Appending returns on ethalon date vector:
date_first = date(1992, 1, 1)
date_last = date(2018, 12, 31)
index_dates = pd.date_range(date_first, date_last, freq = 'B')
ser_ret_index_LOC = pd.Series(np.NaN, index = ser_realized_ret_LOC.index)
dict_realized_ret_LOC = {}
#print('Empty index:', ser_ret_index_LOC.loc['AU', '2002-01-31' : '2007-01-31'].head())
for iter_country in ser_realized_ret_LOC.index.get_level_values(0).unique():
#    if (iter_country == 'AU'):
    ser_ret_index_LOC[iter_country] = (1 + ser_realized_ret_LOC[iter_country]).cumprod()
#        print('Cumprod from Ret:', ser_ret_index_LOC.loc['AU', '2002-01-31' : '2007-01-31'].head())        
    ser_ret_index_LOC[iter_country].iloc[0] = 1      
#        print('Cumprod after reindexation:', ser_ret_index_LOC[iter_country].reindex(index_dates, method = 'ffill').loc['2002-01-31' : '2007-01-31'].head())
    ser_ret_index_LOC_iter = ser_ret_index_LOC[iter_country].reindex(index_dates, method = 'ffill')        
#        print('Cumprod after reindexation and replacement:', ser_ret_index_LOC_iter.loc['2002-01-31' : '2007-01-31'].head())    
    ser_ret_index_LOC_iter.fillna(method = 'ffill', inplace = True)
#        print('Cumprod after fillna:', ser_ret_index_LOC_iter.loc['2002-01-31' : '2007-01-31'].head())    
    ser_realized_ret_LOC_iter = (ser_ret_index_LOC_iter / ser_ret_index_LOC_iter.shift(1) - 1)   
#        print('Ret after decumprod:',(ser_ret_index_LOC_iter / ser_ret_index_LOC_iter.shift(1) - 1).loc['2002-01-31' : '2007-01-31'].head())        
#        print('Ret after decumprod and replacement:', ser_realized_ret_LOC.loc['AU', '2002-01-31' : '2007-01-31'].head())
    dict_realized_ret_LOC[iter_country] = ser_realized_ret_LOC_iter
ser_realized_ret_LOC = pd.concat(dict_realized_ret_LOC)

In [26]:
ser_realized_ret_LOC.loc['AU', '2002-01-31' : '2007-01-31']

AU  2002-01-31    0.000000
    2002-02-01    0.000000
    2002-02-04    0.000000
    2002-02-05    0.000000
    2002-02-06    0.000000
    2002-02-07    0.000000
    2002-02-08    0.000000
    2002-02-11    0.000000
    2002-02-12    0.000000
    2002-02-13    0.000000
    2002-02-14    0.000000
    2002-02-15    0.000000
    2002-02-18    0.000000
    2002-02-19    0.000000
    2002-02-20    0.000000
    2002-02-21    0.000000
    2002-02-22    0.000000
    2002-02-25    0.000000
    2002-02-26    0.000000
    2002-02-27    0.000000
    2002-02-28    0.000000
    2002-03-01    0.000000
    2002-03-04    0.000000
    2002-03-05    0.000000
    2002-03-06    0.000000
    2002-03-07    0.000000
    2002-03-08    0.000000
    2002-03-11    0.000000
    2002-03-12    0.000000
    2002-03-13    0.000000
                    ...   
    2006-12-21    0.000000
    2006-12-22    0.000000
    2006-12-25    0.000000
    2006-12-26    0.000000
    2006-12-27    0.000000
    2006-12-28    0.000000
 